# REPORTS

In [1]:
"""
Creating reports.
"""

# Python built-in modules
import os.path

# Third party libraries
import pandas as pd
from tabulate import tabulate

# Local modules
import crud
import settings
import helpers

## Inventory


In [2]:
def inventory(product:str, output:str):
    """
    Shows inventory.
    """
    pass

In [3]:
    product = "all"
    output = "json"

    ini_rw=settings.ReadWriteINI()
    
    df = pd.read_csv(crud.product_props["path"],delimiter=crud.csv_delimiter, index_col="id")
    df.drop(columns=['sum_costs','sum_revenues'],inplace=True)
    if product == "all":
        report_df = df.sort_values(by='product_name')
        report_title=("Inventory of all products:".upper())
        file_name=f"inventory_all_{ini_rw.get_app_date()}"
    else:
        product_filter = (df['product_name']==product)
        report_df = df.loc[product_filter]
        report_title=(f"Inventory of {product}:".upper())
        file_name=f"inventory_{product}_{ini_rw.get_app_date()}"
    
    if output == 'screen':
        print(report_title)
        print(tabulate(report_df, headers='keys', tablefmt='grid'))
    if output == 'csv':
        file_path = os.path.join(settings.reports_dir,f"{file_name}.csv")
        report_df.to_csv(file_path,sep=crud.csv_delimiter)
    if output == 'json':
        file_path = os.path.join(settings.reports_dir,f"{file_name}.json")
        report_df.to_json(file_path,index=1, orient='records')


## Reports

In [4]:
def report(subject:str, date_range:str, date_type:str, output:str):
    """
    Creates  a report
    """
    pass

In [5]:
    subject = "sales"
    date_range = "2022-1"
    date_type = "quarter"
    output = "screen"

    ini_rw=settings.ReadWriteINI()

    # Creating dataframe.
    if subject=='purchases':
        df = pd.read_csv(crud.purchase_props["path"],delimiter=crud.csv_delimiter, index_col="id")
        date_field = 'date_bought'
        df.drop(columns=['stock_amount','amount_expired', 'expiration_date'],inplace=True)
    if subject=='sales':
        df = pd.read_csv(crud.sale_props["path"],delimiter=crud.csv_delimiter, index_col="id")
        date_field = 'date_sold'

    # Convert string to datetime object.
    df[date_field] = pd.to_datetime(df[date_field],format='%Y-%m-%d')
    
    # Convert string to float.
    df['total_price'] = pd.to_numeric(df['total_price'])

    # Setting varaibles
    if date_type == "day" and date_range=="yesterday":
        iso_date = helpers.date_yesterday(ini_rw.get_app_date())
        report_filter = (df[date_field] == iso_date)
        report_title = f"{subject.capitalize()} on {helpers.long_date_notation(iso_date,'date')}"
        file_name = f"{subject}_{iso_date}"
    if date_type == "day" and date_range=="today":
        iso_date = ini_rw.get_app_date()
        report_filter = (df[date_field] == iso_date)
        report_title = f"{subject.capitalize()} on {helpers.long_date_notation(iso_date,'date')}"
        file_name = f"{subject}_{iso_date}"
    if date_type == "date":
        report_filter = (df[date_field] == date_range)
        report_title = f"{subject.capitalize()} on {helpers.long_date_notation(date_range,'date')}"
        file_name = f"{subject}_{date_range}"
    if date_type == "month":
        year_date = int(date_range[0:4])
        month_date = int(date_range[5:])
        report_filter=(df[date_field].dt.year==year_date) & (df[date_field].dt.month==month_date)
        report_title=f"{subject.capitalize()} in {helpers.long_date_notation(date_range,'month')}"
        file_name = f"{subject}_{date_range}"
    if date_type == "quarter":
        year_date = int(date_range[0:4])
        quarter_date = int(date_range[-1])
        report_filter=(df[date_field].dt.year==year_date) & (df[date_field].dt.quarter==quarter_date)
        report_title=f"{str(subject.capitalize())} in {helpers.long_date_notation(date_range,'quarter')}"
        file_name = f"{subject}_{date_range}"
    if date_type == "year":
        report_filter=(df[date_field].dt.year==date_range)
        report_title=f"{subject.capitalize()} in {date_range}."
        file_name = f"{subject}_{date_range}"

    # Setting filter
    df = df.loc[report_filter]

    df[date_field] = df[date_field].dt.date

    # Create and set format for valuta
    total = df['total_price'].sum()
    df["unit_price"] = df["unit_price"].apply(lambda x: helpers.valuta_notation(float(x)))
    df["total_price"] = df["total_price"].apply(lambda x: helpers.valuta_notation(float(x)))

    
    if output == 'screen':
        print(f"{report_title} - Total of {subject}: {helpers.valuta_notation(float(total))}")
        print(tabulate(df, headers='keys', tablefmt='pretty'))
    if output == 'csv':
        file_path = os.path.join(settings.reports_dir,f"{file_name}.csv")
        df.to_csv(file_path,sep=crud.csv_delimiter)
    if output == 'json':
        file_path = os.path.join(settings.reports_dir,f"{file_name}.json")
        df.to_json(file_path,index=1, orient='records')

    

Sales in Q1 of 2022 - Total of sales: € 6.70
+------------------+------------------+--------------+-------------+------------+------------+-------------+
|        id        |    product_id    | product_name | amount_sold | date_sold  | unit_price | total_price |
+------------------+------------------+--------------+-------------+------------+------------+-------------+
| S.20220207.00001 | P.20220207.00001 |    apples    |     25      | 2022-02-07 |   € 0.13   |   € 3.25    |
| S.20220207.00002 | P.20220207.00002 |   oranges    |     15      | 2022-02-07 |   € 0.23   |   € 3.45    |
+------------------+------------------+--------------+-------------+------------+------------+-------------+
